In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day32-binning-and-binarization/train.csv')

In [3]:
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [4]:
df.dropna(inplace=True)

In [5]:
df1 = df[['Survived','Parch','SibSp','Age','Fare']]

In [6]:
df1.head()

Survived  Parch  SibSp   Age     Fare
1          1      0      1  38.0  71.2833
3          1      0      1  35.0  53.1000
6          0      0      0  54.0  51.8625
10         1      1      1   4.0  16.7000
11         1      0      0  58.0  26.5500

In [7]:
df1['family'] = df1['SibSp'] + df1['Parch']

C:\Users\mynam\AppData\Local\Temp/ipykernel_1680/1848115773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['family'] = df1['SibSp'] + df1['Parch']


In [8]:
df1.head()

Survived  Parch  SibSp   Age     Fare  family
1          1      0      1  38.0  71.2833       1
3          1      0      1  35.0  53.1000       1
6          0      0      0  54.0  51.8625       0
10         1      1      1   4.0  16.7000       2
11         1      0      0  58.0  26.5500       0

In [9]:
df1.drop(columns=['Parch','SibSp'],inplace=True)

C:\Users\mynam\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [10]:
df1.head()

Survived   Age     Fare  family
1          1  38.0  71.2833       1
3          1  35.0  53.1000       1
6          0  54.0  51.8625       0
10         1   4.0  16.7000       2
11         1  58.0  26.5500       0

In [11]:
X = df1.drop(columns='Survived')
y = df1['Survived']

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
X_train.head()

Age     Fare  family
331  45.5  28.5000       0
336  29.0  66.6000       1
193   3.0  26.0000       2
75   25.0   7.6500       0
248  37.0  52.5542       2

In [17]:
X_test.head()

Age      Fare  family
118  24.0  247.5208       1
251  29.0   10.4625       2
742  21.0  262.3750       4
544  50.0  106.4250       1
712  48.0   52.0000       1

In [14]:
# without Binarizer
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.5675675675675675

In [15]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

0.6181286549707602

In [16]:
from sklearn.preprocessing import Binarizer

In [21]:
# Applynig Binarizer 
trf = ColumnTransformer([('bina',Binarizer(copy=False),[2])],remainder='passthrough')

In [22]:
X_train_transformed = trf.fit_transform(X_train)
X_test_transformed = trf.transform(X_test)

In [24]:
pd.DataFrame(X_train_transformed,columns=['Family','Age','Fare'])

Family   Age      Fare
0       0.0  45.5   28.5000
1       1.0  29.0   66.6000
2       1.0   3.0   26.0000
3       0.0  25.0    7.6500
4       1.0  37.0   52.5542
..      ...   ...       ...
141     0.0  30.0   93.5000
142     1.0  46.0   61.1750
143     0.0  48.0   26.5500
144     0.0  33.0    5.0000
145     1.0  18.0  108.9000

[146 rows x 3 columns]

In [25]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)
y_pred = clf.predict(X_test_transformed)
accuracy_score(y_test,y_pred)

0.5945945945945946

In [27]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

0.6403508771929826